In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import colorcet

import sys

sys.path.append("../../")
from _plotly_templates.custom_template import custom_template

pio.templates.default = custom_template
pio.renderers.default = "png"


In [4]:
entsoe_data = (
    pd.read_pickle("../../_data/entsoe/entsoe_DE_AT_cty_engineered_1h")
    .rolling(720, center=False)
    .mean()["2017-10-1":"2019-09-30"]
)

In [ ]:
# Read data
pos_afrr_price_data = (
    pd.read_pickle("../_data/regelleistungnet/pos_afrr_price_data.pkl")
    .astype("float")
    .sort_index()["2018-10-16":"2020-07-30"]
)
neg_afrr_price_data = (
    pd.read_pickle("../_data/regelleistungnet/neg_afrr_price_data.pkl")
    .astype("float")
    .sort_index()["2018-10-16":"2020-07-30"]
)

afrr_price_data = pd.concat([pos_afrr_price_data, neg_afrr_price_data], axis=1)
afrr_price_data = afrr_price_data.resample("4H").ffill()
# afrr_price_data = afrr_price_data.tz_localize('CET')

afrr_price_data["neg_factor"] = 0
afrr_price_data["neg_factor"]["2018-10-16":"2018-12-31"] = 0.0607
afrr_price_data["neg_factor"]["2019-1-1":"2019-3-31"] = 0.0675
afrr_price_data["neg_factor"]["2019-4-1":"2018-6-30"] = 0.078
afrr_price_data["neg_factor"]["2019-7-1":"2019-7-31"] = 0.0841

afrr_price_data["pos_factor"] = 0
afrr_price_data["pos_factor"]["2018-10-16":"2018-12-31"] = 0.073
afrr_price_data["pos_factor"]["2019-1-1":"2019-3-31"] = 0.0761
afrr_price_data["pos_factor"]["2019-4-1":"2018-6-30"] = 0.0781
afrr_price_data["pos_factor"]["2019-7-1":"2019-7-31"] = 0.0799

afrr_price_data["neg_avg_mpv_price"] = (
    afrr_price_data["neg_avg_cap_price"]
    - afrr_price_data["neg_avg_en_price"] * afrr_price_data["neg_factor"]
)
afrr_price_data["pos_avg_mpv_price"] = (
    afrr_price_data["pos_avg_cap_price"]
    + afrr_price_data["pos_avg_en_price"] * afrr_price_data["pos_factor"]
)

In [ ]:
fig = make_subplots(
    rows=2,
    cols=2,
    column_widths=[0.66, 0.34],
    specs=[
        [{"type": "scatter"}, {"type": "table"}],
        [{"type": "scatter"}, {"type": "table"}],
    ],
    vertical_spacing=0.15,
    horizontal_spacing=0.01,
)

fig.add_trace(
    go.Scatter(
        x=entsoe_data.index,
        y=entsoe_data.price_day_ahead_AT,
        name="AT",
        line_color=px.colors.qualitative.T10[2],
    ),
    row=1,
    col=1,
)
fig.add_trace(
    go.Scatter(
        x=entsoe_data.index,
        y=entsoe_data.price_day_ahead_DE,
        name="DE",
        line_color=px.colors.qualitative.T10[0],
    ),
    row=1,
    col=1,
)
fig.add_vline(
    x="2018-10-1", line_width=2, line_dash="dash", line_color="black", row=1, col=1
)
fig.add_trace(
    go.Scatter(
        x=afrr_price_data.index,
        y=afrr_price_data.neg_avg_mpv_price,
        showlegend=False,
        line_color=px.colors.qualitative.T10[0],
    ),
    row=2,
    col=1,
)
fig.add_vline(
    x="2019-07-31", line_width=2, line_dash="dash", line_color="black", row=2, col=1
)

fig.update_yaxes(title_text="Day-ahead<br>Electricity Price<br>[€/MWh]", 
    range=[20, 70], row=1, col=1)


fig.add_trace(
    go.Table(
        header_values=["<b>Features:</b>"],
        header_font_size=24,
        header_line_width=0,
        header_align="left",
        header_height=40,
        cells_values=[
            [
                # "    ",
                "    Austrian residual load",
                "    Swiss residual load",
                "    Czech residual load",
                "    German residual load",
                "    Hungarian residual load",
                "    Italian residual load",
                "    Slovenian residual load",
            ]
        ],
        cells_align="left",
        cells_line_width=0,
        cells_height=33,
    ),
    row=1,
    col=2,
)

fig.add_trace(
    go.Table(
        header_values=["<b>Features:</b>"],
        header_font_size=24,
        header_line_width=0,
        header_align="left",
        header_height=2,
        cells_values=[
            [
            "    Hour",
            "    Weekday",
            "  <b>Generation per type:</b>",
            "    Biomass",
            "    Gas",
            "    Hard coal",
            "    Lignite",
            "    Nuclear"],
            [
            "Load",
            "",
            "",
            "Pumped hydro",
            "Reservoir hydro",
            "ROR hydro",
            "Solar",
            "Wind",
            
            ]
        ],
        cells_align="left",
        cells_line_width=0,
        cells_height=27,
        cells_font_size=20,
    ),
    row=2,
    col=2,
)


fig.update_yaxes(
    title_text="Negative aFRR<br>Average Auction Price<br>[€]", 
    range=[-5, 170],
    row=2, col=1
)

fig.update_xaxes(
    tickvals=["2018", "2018-10", "2019"],
    ticktext=["2018", "Oct 1, 2018", "2019"],
    row=1,
    col=1,
)
fig.update_xaxes(
    tickvals=["2019", "2019-07-31", "2020"],
    ticktext=["2019", "Jul 31, 2019", "2020"],
    row=2,
    col=1,
)


fig.update_layout(font=dict(family="Libertine", size=24), width=1575, height=770)
fig.update_annotations(font=dict(family="Libertine", size=24))
fig.update_layout(legend_y=0.99, legend_x=0.64, legend_xanchor='right', margin_l=175, margin_r=36)

fig.add_annotation(
    x=-0.12,
    y=1.2,
    xref="paper",
    yref="y domain",
    text="<b>(a)</b>",
    showarrow=False,
    font=dict(family="Libertine", size=45),
)
fig.add_annotation(
    x=-0.12,
    y=1.2,
    xref="paper",
    yref="y2 domain",
    text="<b>(b)</b>",
    showarrow=False,
    font=dict(family="Libertine", size=45),
)

pio.write_image(
        fig,
        "../figures/fig_1.pdf",
        format="pdf",
        validate=False,
        engine="kaleido",
    )
